In [1]:
import re
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt


In [2]:
# # Set pandas options to display all columns and rows
# pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.max_rows', None)     # Show all rows

## Looping through scraped data from 100X100trail

In [3]:
races = ['TOR330'
#         ,'TOR450'
        ]
years = [ '2021',
        '2022',
         '2023', 
    '2024'
        ]

TORX_df = {}

for race in races:
    for year in years:
        try:
            df = pd.read_excel(f'{race} Data/clean_{race}_aid_stations_{year}.xlsx',
                                     dtype={'Start Date': 'string',
                                            'Year': 'string'})
            print(f'{race}_{year} {df.shape}')
            # Store the DataFrame in the dictionary with a key like 'TOR330_2021'
            TORX_df[f'{race}_{year}'] = df
        except:
            pass
        
        print('*'*50),

**************************************************
TOR330_2022 (960, 91)
**************************************************
TOR330_2023 (1209, 92)
**************************************************
TOR330_2024 (1098, 92)
**************************************************


In [4]:
TORX = pd.concat(TORX_df)

In [5]:
TORX.columns

Index(['Unnamed: 0', 'Name', 'Bib', 'Sex', 'Nationality', 'Year', 'Race',
       'Category', 'Status', 'Status1', 'Wave', 'Duration', 'RITIRIO',
       'Retired', 'Retired_Section', 'new_Start Date', 'Baite Youlaz',
       'La Thuile', 'Rifugio Deffeyes', 'Planaval', 'Valgrisenche IN',
       'Valgrisenche OUT', 'Chalet Epee', 'Rhemes-Notre-Dame', 'Eaux Rousse',
       'Rifugio Sella', 'Cogne IN', 'Cogne OUT', 'Goilles', 'Rifugio Dondena',
       'Chardonney', 'Pontboset', 'Donnas IN', 'Donnas OUT', 'Perloz', 'Sassa',
       'Rifugio Coda', 'Rifugio della Barma', 'Lago Chiaro',
       'Col della Vecchia', 'Niel La Gruba', 'Loo', 'Gressoney IN',
       'Gressoney OUT', 'Rifugio Alpenzu', 'Champoluc',
       'Rifugio Grand Tournalin', 'Valtournenche IN', 'Valtournenche OUT',
       'Vareton', 'Rifugio Magià', 'Rifugio Cuney', 'Bivacco R. Clermont',
       'Oyace', 'Bruson Arp', 'Ollomont IN', 'Ollomont OUT',
       'Rifugio Champillon', 'Ponteille Desot', 'Bosses', 'Rifugio Frassati',
  

### Converting to seconds for Tableau Analysis

In [6]:
TORX_dem = TORX[['Name', 'Bib', 'Sex', 'Nationality', 'Year', 'Race', 
      'Category', 'Status', 'Status1', 'Wave', 
      'RITIRIO', 'Retired', 'Retired_Section', 'Duration_in_seconds']].reset_index(drop = True)
      
TORX_times = TORX[[ 'Bib','Year', 'Race',
      'Section 1 Time_in_seconds', 'Time Spent in Valgrisenche_in_seconds', 
      'Section 2 Time_in_seconds', 'Time Spent in Cogne_in_seconds', 
        'Section 3 Time_in_seconds','Time Spent in Donnas_in_seconds', 
        'Section 4 Time_in_seconds','Time Spent in Gressoney_in_seconds', 
        'Section 5 Time_in_seconds', 'Time Spent in Valtournenche_in_seconds', 
        'Section 6 Time_in_seconds','Time Spent in Ollomont_in_seconds', 
        'Section 7 Time_in_seconds']].reset_index(drop = True)

In [7]:
TORX_dem['Retired_Section'].unique()

array([nan, 'Section 2', 'Section 4', 'Section 3', 'Section 7',
       'Section 6', 'Section 1', 'Section 5', 'DNS'], dtype=object)

In [8]:
TORX_dem.groupby(['Retired_Section'])['Retired_Section'].size()

Retired_Section
DNS          131
Section 1    238
Section 2    409
Section 3     96
Section 4    388
Section 5     72
Section 6    108
Section 7     34
Name: Retired_Section, dtype: int64

In [9]:
TORX_dem['Retired_Section'].unique()

array([nan, 'Section 2', 'Section 4', 'Section 3', 'Section 7',
       'Section 6', 'Section 1', 'Section 5', 'DNS'], dtype=object)

In [10]:
# Unpivot the section-related columns
value_vars = [
    'Section 1 Time_in_seconds', 'Time Spent in Valgrisenche_in_seconds', 
    'Section 2 Time_in_seconds', 'Time Spent in Cogne_in_seconds', 
    'Section 3 Time_in_seconds', 'Time Spent in Donnas_in_seconds',
    'Section 4 Time_in_seconds', 'Time Spent in Gressoney_in_seconds', 
    'Section 5 Time_in_seconds', 'Time Spent in Valtournenche_in_seconds', 
    'Section 6 Time_in_seconds', 'Time Spent in Ollomont_in_seconds', 
    'Section 7 Time_in_seconds'
]

# Melting the dataset
TORX_times_pivoted = pd.melt(
    TORX_times, 
    id_vars=[ 'Bib', 'Year', 'Race'], 
    value_vars=value_vars, 
    var_name='Stage', 
    value_name='Duration'
)


TORX_times_pivoted['Stage'] = TORX_times_pivoted['Stage'].str.replace(' Time_in_seconds', '')
# Display the unpivoted dataset
print(TORX_times_pivoted)

        Bib  Year    Race      Stage  Duration
0         3  2022  TOR330  Section 1   26648.0
1         9  2022  TOR330  Section 1   27065.0
2         5  2022  TOR330  Section 1   28919.0
3        12  2022  TOR330  Section 1   28916.0
4       275  2022  TOR330  Section 1   30578.0
...     ...   ...     ...        ...       ...
42466  1055  2024  TOR330  Section 7       NaN
42467   525  2024  TOR330  Section 7       NaN
42468  1007  2024  TOR330  Section 7       NaN
42469  1464  2024  TOR330  Section 7       NaN
42470  1313  2024  TOR330  Section 7       NaN

[42471 rows x 5 columns]


In [11]:
TORX_dem.groupby(['Retired_Section', 'Retired' ])['Retired_Section'].count()

Retired_Section  Retired                
DNS              new_Start Date             131
Section 1        Baite Youlaz                 2
                 La Thuile                   34
                 Planaval                    24
                 Rifugio Deffeyes            38
                 Valgrisenche               140
Section 2        Chalet Epee                 25
                 Cogne                      109
                 Eaux Rousse                156
                 Rhemes-Notre-Dame          100
                 Rifugio Sella               19
Section 3        Chardonney                  15
                 Donnas                      57
                 Goilles                      4
                 Pontboset                    5
                 Rifugio Dondena             15
Section 4        Col della Vecchia           28
                 Gressoney                  135
                 Lago Chiaro                 16
                 Loo                         16

### Saving for Tableau

In [12]:
TORX_dem.to_excel(f'TOR330 Data/clean_TORX_dem.xlsx' , index = False)
TORX_times_pivoted.to_excel(f'TOR330 Data/clean_TORX_times.xlsx' , index = False)